# Text Classification

We are using Natural Language Processing with Disaster Tweets Kaggle dataset. The data is comprised of tweets, and our objective is to forecast which tweets are associated with a tragedy. This may enhance reaction times for a variety of interested parties, including police forces, fire departments, and news organisations, among others. We will do text categorization using predictive machine learning models, which is a subset of natural language processing.

In [ ]:
import pyspark
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-3.0.3-bin-hadoop2.7'

Starting the Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NLP").getOrCreate()

Import Important modules required 

In [ ]:
from pyspark.ml import Pipeline 
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import col, udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Now we are loading the dataset uci-news-aggregator.csv.

In [ ]:
#read news csv dataset from the working directory
news_data = spark.read.csv('uci-news-aggregator.csv',header= True)
news_data.printSchema()
news_data.show()
#news_data = news_data.limit(300)
#news_data.cache

root
 |-- ID: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- PUBLISHER: string (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- STORY: string (nullable = true)
 |-- HOSTNAME: string (nullable = true)
 |-- TIMESTAMP: string (nullable = true)

+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
| ID|               TITLE|                 URL|           PUBLISHER|CATEGORY|               STORY|            HOSTNAME|    TIMESTAMP|
+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
|  1|Fed official says...|http://www.latime...|   Los Angeles Times|       b|ddUyU0VZz0BRneMio...|     www.latimes.com|1394470370698|
|  2|Fed's Charles Plo...|http://www.livemi...|            Livemint|       b|ddUyU0VZz0BRneMio...|    www.livemint.com|1394470371207|
|  3|US open: Stock

<bound method DataFrame.cache of DataFrame[ID: string, TITLE: string, URL: string, PUBLISHER: string, CATEGORY: string, STORY: string, HOSTNAME: string, TIMESTAMP: string]>

We can check the count of totalitems in the dataset for analysis

In [ ]:
#count data items present in the set
news_data.count()

300

We are selecting the titles of tweets and the corresponding category of each tweet

In [ ]:
#select the titles of tweets corresponding to category
title_category = news_data.select("TITLE","CATEGORY")
title_category.show()

+--------------------+--------+
|               TITLE|CATEGORY|
+--------------------+--------+
|Fed official says...|       b|
|Fed's Charles Plo...|       b|
|US open: Stocks f...|       b|
|Fed risks falling...|       b|
|Fed's Plosser: Na...|       b|
|Plosser: Fed May ...|       b|
|Fed's Plosser: Ta...|       b|
|Fed's Plosser exp...|       b|
|US jobs growth la...|       b|
|ECB unlikely to e...|       b|
|ECB unlikely to e...|       b|
|EU's half-baked b...|       b|
|Europe reaches cr...|       b|
|ECB FOCUS-Stronge...|       b|
|EU aims for deal ...|       b|
|Forex - Pound dro...|       b|
|Noyer Says Strong...|       b|
|EU Week Ahead Mar...|       b|
|ECB member Noyer ...|       b|
|Euro Anxieties Wa...|       b|
+--------------------+--------+
only showing top 20 rows



This is the custom function definition to count the null values

In [ ]:
#definition to count the null values
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

We are applying the custom function to the data frame title_category

In [ ]:
null_columns_count_list = null_value_count(title_category)
#spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

# Cleaning the dataset

Now we can drop the null values

In [ ]:
#Drop null and not available values
title_category = title_category.dropna()
title_category.count()
title_category.show(truncate=False)

+---------------------------------------------------------------------------+--------+
|TITLE                                                                      |CATEGORY|
+---------------------------------------------------------------------------+--------+
|Fed official says weak data caused by weather, should not slow taper       |b       |
|Fed's Charles Plosser sees high bar for change in pace of tapering         |b       |
|US open: Stocks fall after Fed official hints at accelerated tapering      |b       |
|Fed risks falling 'behind the curve', Charles Plosser says                 |b       |
|Fed's Plosser: Nasty Weather Has Curbed Job Growth                         |b       |
|Plosser: Fed May Have to Accelerate Tapering Pace                          |b       |
|Fed's Plosser: Taper pace may be too slow                                  |b       |
|Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014   |b       |
|US jobs growth last month hit by weather:F

We are counting the distinct casses from the category column

In [ ]:
#count the distinct category available in the dataset
title_category.select("Category").distinct().count()
title_category.groupBy("Category").count().orderBy(col("count").desc()).show(truncate=False)

+--------+-----+
|Category|count|
+--------+-----+
|b       |299  |
|GigaOM  |1    |
+--------+-----+



In [ ]:
title_category.groupBy("TITLE").count().orderBy(col("count").desc()).show(truncate=False)

+---------------------------------------------------------------------------+-----+
|TITLE                                                                      |count|
+---------------------------------------------------------------------------+-----+
|Fed official says weak data caused by weather, should not slow taper       |1    |
|Fed's Charles Plosser sees high bar for change in pace of tapering         |1    |
|US open: Stocks fall after Fed official hints at accelerated tapering      |1    |
|Fed risks falling 'behind the curve', Charles Plosser says                 |1    |
|Fed's Plosser: Nasty Weather Has Curbed Job Growth                         |1    |
|Plosser: Fed May Have to Accelerate Tapering Pace                          |1    |
|Fed's Plosser: Taper pace may be too slow                                  |1    |
|Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014   |1    |
|US jobs growth last month hit by weather:Fed President Charles Plosser     

Now let us remove the numbers present in the tweets

In [ ]:
#clean numbers and other unwanted characters from the tweets
title_category = title_category.withColumn("only_str",regexp_replace(col('TITLE'), '\d+', ''))
title_category.select("TITLE","only_str").show(truncate=False)

+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
|TITLE                                                                      |only_str                                                                   |
+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
|Fed official says weak data caused by weather, should not slow taper       |Fed official says weak data caused by weather, should not slow taper       |
|Fed's Charles Plosser sees high bar for change in pace of tapering         |Fed's Charles Plosser sees high bar for change in pace of tapering         |
|US open: Stocks fall after Fed official hints at accelerated tapering      |US open: Stocks fall after Fed official hints at accelerated tapering      |
|Fed risks falling 'behind the curve', Charles Plosser says                 

Split the text into constituent words

In [ ]:
#split the text to tokens using tokenizer
#https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.feature.RegexTokenizer.html
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(title_category)
raw_words.show()

+--------------------+--------+--------------------+--------------------+
|               TITLE|CATEGORY|            only_str|               words|
+--------------------+--------+--------------------+--------------------+
|Fed official says...|       b|Fed official says...|[fed, official, s...|
|Fed's Charles Plo...|       b|Fed's Charles Plo...|[fed, s, charles,...|
|US open: Stocks f...|       b|US open: Stocks f...|[us, open, stocks...|
|Fed risks falling...|       b|Fed risks falling...|[fed, risks, fall...|
|Fed's Plosser: Na...|       b|Fed's Plosser: Na...|[fed, s, plosser,...|
|Plosser: Fed May ...|       b|Plosser: Fed May ...|[plosser, fed, ma...|
|Fed's Plosser: Ta...|       b|Fed's Plosser: Ta...|[fed, s, plosser,...|
|Fed's Plosser exp...|       b|Fed's Plosser exp...|[fed, s, plosser,...|
|US jobs growth la...|       b|US jobs growth la...|[us, jobs, growth...|
|ECB unlikely to e...|       b|ECB unlikely to e...|[ecb, unlikely, t...|
|ECB unlikely to e...|       b|ECB unl

Remove the stop words from segregated list of words

In [ ]:
#Remove and segregate stop words form the word list like for, by, in etc.
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StopWordsRemover.html
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = remover.transform(raw_words)
words_df.select("words","filtered").show(truncate=False)

+-------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+
|words                                                                                |filtered                                                                       |
+-------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+
|[fed, official, says, weak, data, caused, by, weather, should, not, slow, taper]     |[fed, official, says, weak, data, caused, weather, slow, taper]                |
|[fed, s, charles, plosser, sees, high, bar, for, change, in, pace, of, tapering]     |[fed, charles, plosser, sees, high, bar, change, pace, tapering]               |
|[us, open, stocks, fall, after, fed, official, hints, at, accelerated, tapering]     |[us, open, stocks, fall, fed, official, hints, accelerated, tapering]    

The category column in the dataframe can now be mapped to categoryIndex

In [ ]:
#Index the string for different category
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StringIndexer.html
indexer = StringIndexer(inputCol="CATEGORY", outputCol="categoryIndex")
feature_data = indexer.fit(words_df).transform(words_df)
feature_data.select("CATEGORY","categoryIndex").show()

+--------+-------------+
|CATEGORY|categoryIndex|
+--------+-------------+
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
|       b|          0.0|
+--------+-------------+
only showing top 20 rows



Convert text into vectors of token counts

In [ ]:
#converting text to vectors and count the tokens
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(feature_data)
countVectorizer_feateures = model.transform(feature_data)

# Partition the dataset into training and test datasets


In [ ]:
#partition the data set into training and test sets
(trainingData, testData) = countVectorizer_feateures.randomSplit([0.8, 0.2],seed = 11)
trainingData.show()
testData.show()

+--------------------+--------+--------------------+--------------------+--------------------+-------------+--------------------+
|               TITLE|CATEGORY|            only_str|               words|            filtered|categoryIndex|            features|
+--------------------+--------+--------------------+--------------------+--------------------+-------------+--------------------+
|"MtGox ""fraud ev...|  GigaOM|"MtGox ""fraud ev...|[mtgox, fraud, ev...|[mtgox, fraud, ev...|          1.0|(625,[24,38,54,70...|
|$5 20-piece chick...|       b|$ -piece chicken ...|[piece, chicken, ...|[piece, chicken, ...|          0.0|(625,[2,7,10,125,...|
|'Hacked docs' pro...|       b|'Hacked docs' pro...|[hacked, docs, pr...|[hacked, docs, pr...|          0.0|(625,[22,24,28,30...|
|10 Things You Nee...|       b| Things You Need ...|[things, you, nee...|[things, need, kn...|          0.0|(625,[155,171,178...|
|10 Things You Nee...|       b| Things You Need ...|[things, you, nee...|[things, need, kn

# Model Training and Prediction

## Naive Bayes Model

In [ ]:
#https://scikit-learn.org/stable/modules/naive_bayes.html
nb = NaiveBayes(modelType="multinomial",labelCol="categoryIndex", featuresCol="features")
nbModel = nb.fit(trainingData)
nb_predictions = nbModel.transform(testData)

In [ ]:
nb_predictions.select("prediction", "categoryIndex", "features").show(100)

+----------+-------------+--------------------+
|prediction|categoryIndex|            features|
+----------+-------------+--------------------+
|       0.0|          0.0|(625,[36,145,409]...|
|       0.0|          0.0|(625,[2,7,10,155,...|
|       0.0|          0.0|(625,[14,55,57,64...|
|       0.0|          0.0|(625,[0,1,8,18,28...|
|       0.0|          0.0|(625,[0,1,3,18,23...|
|       0.0|          0.0|(625,[0,1,3,8,28,...|
|       0.0|          0.0|(625,[3,165,311,3...|
|       0.0|          0.0|(625,[0,1,2,3,13,...|
|       0.0|          0.0|(625,[3,13,18,47,...|
|       0.0|          0.0|(625,[3,36,68,252...|
|       0.0|          0.0|(625,[11,46,67,24...|
|       0.0|          0.0|(625,[4,11,46,426...|
|       0.0|          0.0|(625,[19,81,97,10...|
|       0.0|          0.0|(625,[0,1,3,86,30...|
|       0.0|          0.0|(625,[4,11,83,99,...|
|       0.0|          0.0|(625,[6,9,27,34,6...|
|       0.0|          0.0|(625,[6,9,27,34,4...|
|       0.0|          0.0|(625,[6,9,27,3

In [ ]:
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="accuracy")
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))
print("Test Error of NaiveBayes = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is = 0.983333
Test Error of NaiveBayes = 0.0166667 


## Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
## Fitting the model
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
lr = LogisticRegression(featuresCol = 'features', labelCol = 'categoryIndex', maxIter=10)
lrModel = lr.fit(trainingData)
lrPreds = lrModel.transform(testData)

In [ ]:
## Evaluating the model
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lrPreds)
print("Accuracy of Logistic Regression is = %g"% (lr_accuracy))

Accuracy of Logistic Regression is = 1


## Decision Tree Model

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
## Fitting the model
#https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'categoryIndex', maxDepth = 3)
dtModel = dt.fit(trainingData)
dtPreds = dtModel.transform(testData)

In [ ]:
## Evaluating the model
#https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/ml/evaluation/MulticlassClassificationEvaluator.html
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(dtPreds)
print("Accuracy of Decision Trees is = %g"% (dt_accuracy))
#Accuracy of Decision Trees is = 0.651034

Accuracy of Decision Trees is = 1
